<img src="https://www.austral.edu.ar/wp-content/uploads/2022/10/facultades-horizontales-03.png" width="50%" alt="Universidad Austral - Facultad de Ingeniería">

# Maestría en Ciencia de Datos 2024/2025

## Simulación y OPtimización en Ciencia de Datos 
## Trabajo Práctico Integral

Profesores:

- DEL ROSSO, Rodrigo
- NUSKE, Ezequiel

Integrantes:

-	NICOLAU, Jorge

Ver [RESOLUCIÓN](#resolucion)

### Introducción

Este Trabajo Práctico (TP) integra los contenidos vistos en las clases

- **Unidad I** – Generación de números pseudoaleatorios y Monte Carlo
- **Unidad II** – Bayes, cadenas de Markov y Metropolis–Hastings
- **Unidad III** – Simulación de eventos discretos (SED)
- **Unidad IV** – Procesos continuos (NHPP, CTMC, SDE)
- **Unidad V** – Reacciones químicas estocásticas: Gillespie SSA y Next Reaction Method
 
El objetivo es que el alumno implemente técnicas de simulación, compare métodos, valide sus resultados y presente visualizaciones claras.

Cada apartado incluye consignas, código sugerido y entregables obligatorios.


### Parte 1 - Generación de Números Pseudoaleatorios y Monte Carlo

### 1.1 Implementación de un LCG

Implementar un Generador Congruencial Lineal (LCG):

<center>
$X_{n+1} = (aX_{n}+c)\text{ mod }m$
</center>

**Parámetros obligatorios**

- Un set “bueno”
- Uno “malo” (ej., RANDU)

In [1]:
# Código base

def lcg(n, seed=123, a=1103515245, c=12345, m=2**31):
    x = seed
    seq = []
    for _ in range(n):
        x = (a * x + c) % m
        seq.append(x/m)
    return seq

print(lcg(10))

[0.20531828328967094, 0.6873863865621388, 0.7767890496179461, 0.4024707484059036, 0.5324797946959734, 0.10148253152146935, 0.6351402262225747, 0.34936570515856147, 0.7226534709334373, 0.027218241710215807]


Entregables

- Histogramas
- Runs Test
- Gráfico de triples ($u_{i}, u_{i+1}, u_{i+2}$)

#### 1.2 Estimación de $\pi$ por Monte Carlo

Simular puntos en el cuadrado $\left[ 0,1 \right]^{2}$ y estimar:

<center>
$\pi \approx 4 \cdot \text{mean}\left( x^{2}+y^{2} \le 1\right)$
</center>

In [2]:
# Código base

import numpy as np

N = 100000
u1 = np.random.uniform(0, 1, N)
u2 = np.random.uniform(0, 1, N)
pi_est = 4 * np.mean(u1**2 + u2**2 <= 1)

print(pi_est)

3.13864


### Parte 2 - Metropolis–Hastings y Bayesian Inference

#### 2.1 Posterior beta analítica
Sea una moneda con 10 lanzamientos y 7 caras.

<center>
$\text{Posterior}=\text{Beta}\left( 8,4 \right)$
</center>

#### 2.2 Implementación MH

In [3]:
# Código base

import numpy as np
from scipy.stats import binom

def mh(n_iter=5000, start=0.5, prop_sd=0.1):
    # Inicializar la cadena (array lleno de ceros)
    theta = np.zeros(n_iter)
    theta[0] = start
    
    # Bucle desde el segundo elemento (índice 1) hasta el final
    for t in range(1, n_iter):
        # Propuesta: Distribución normal centrada en el theta anterior
        # Equivalente en R: rnorm(1, theta[t-1], prop_sd)
        prop = np.random.normal(loc=theta[t-1], scale=prop_sd)
        
        # Verificación de límites (Prior Uniforme Implícito [0,1])
        # Si la propuesta se sale del rango [0, 1], se rechaza inmediatamente
        if prop < 0 or prop > 1:
            theta[t] = theta[t-1]
        else:
            # Calcular la Verosimilitud (Likelihood) usando la función de masa de probabilidad
            # Equivalente en R: dbinom(7, 10, prob)
            num = binom.pmf(k=7, n=10, p=prop) * 1      # * 1 representa el Prior (Uniforme)
            den = binom.pmf(k=7, n=10, p=theta[t-1]) * 1
            
            # Probabilidad de aceptación (alpha)
            ratio = num / den
            alpha = min(1, ratio)
            
            # Paso de Aceptación o Rechazo
            # Generamos un número aleatorio uniforme y lo comparamos con alpha
            if np.random.uniform() < alpha:
                theta[t] = prop     # Aceptar la propuesta
            else:
                theta[t] = theta[t-1] # Rechazar y mantener el valor anterior
                
    return theta

chain = mh()
print(f"Media Posterior Estimada: {np.mean(chain):.4f}")

Media Posterior Estimada: 0.6649


**Diagnósticos obligatorios**

- Traceplot
- Autocorrelación
- Histograma vs Beta(8,4)
- R-hat con 4 cadenas
- ESS

### Parte 3 — Simulación de Eventos Discretos (M/M/1 o M/M/c)

Simular durante 8 horas un sistema de colas con:

- Llegadas Poisson: $\lambda$ = 10 / hora
- Servicio exponencial: $\lambda$ = 4 / hora
- Opcional: c servidores

Se debe implementar LEF (Lista de Eventos Futuros).

In [4]:
# Codigo base 

import numpy as np

def simulate_mm1(lam, mu, T_max):
    # Nota: 'lambda' es una palabra reservada en Python, usamos 'lam'
    t = 0.0
    n = 0
    
    # R usa la tasa (rate) para rexp: rexp(1, lambda)
    # Numpy usa la escala (scale = 1/rate): exponential(1/lam)
    t_arr = np.random.exponential(scale=1/lam)
    t_dep = float('inf')  # Infinito en Python
    
    arrivals = 0
    departures = 0
    wait_times = []  # En Python usamos listas dinámicas en lugar de vectores c()
    
    while t < T_max:
        if t_arr < t_dep:
            # --- Evento de Llegada ---
            t = t_arr
            n += 1
            arrivals += 1
            
            # Si el servidor estaba libre (n=1 tras la llegada), programamos servicio
            if n == 1:
                t_dep = t + np.random.exponential(scale=1/mu)
            
            # Programar la siguiente llegada
            t_arr = t + np.random.exponential(scale=1/lam)
            
        else:
            # --- Evento de Salida ---
            t = t_dep
            n -= 1
            departures += 1
            wait_times.append(t_dep) # Añadimos el tiempo a la lista
            
            if n > 0:
                # Si quedan clientes, programar la siguiente salida
                t_dep = t + np.random.exponential(scale=1/mu)
            else:
                # Si no hay nadie, la próxima salida es "infinita"
                t_dep = float('inf')
    
    # Retornamos un diccionario (equivalente a la lista nombrada de R)
    return {
        "arrivals": arrivals, 
        "departures": departures, 
        "wait_times": wait_times
    }

# Lambda = 2 clientes/min, Mu = 3 clientes/min, Tiempo = 100 min
res = simulate_mm1(lam=2, mu=3, T_max=100)

print(f"Llegadas totales: {res['arrivals']}")
print(f"Salidas totales: {res['departures']}")
print(f"Últimos 5 tiempos de salida: {res['wait_times'][-5:]}")

Llegadas totales: 200
Salidas totales: 200
Últimos 5 tiempos de salida: [97.77449079918325, 97.77824533692458, 98.87566343900198, 99.58589583917923, 100.01773726551271]


### Parte 4 — Modelos Continuos (Elegir uno)

#### 4.1 Opción A: NHPP por Thinning

<center>
$\lambda(t)=5+3\text{ sin}\left( \frac{2\pi t}{24} \right)$
</center>

Simular 7 días (168 horas)

#### 4.2 Opción B: CTMC
Matriz de tasas:

| From | To | Rate |
|----------|----------|----------|
| A    | B   | 0.4   |
| A    | C   | 0.2   |
| B    | A   | 0.1   |
| C    | A   | 0.3   |

Simular 2000 saltos.

#### 4.3 Opción C: SDE (GBM)

<center>
$\text{d}S_{t}=0.05S_{t}dt+0.2S_{t}dW_{t}$
</center>

Simular 1000 trayectorias

###  Parte 5 — Gillespie SSA o Next Reaction Method
Sistema:

- $\to \text{mRNA}$ con tasa ($k_{1}=10$)
- $\text{mRNA}\to $ con tasa ($k_{2}=1$)

In [5]:
# Codigo base

import numpy as np
import pandas as pd

def gillespie(Tmax, k1=10, k2=1):
    t = 0.0
    X = 0
    
    # Listas para almacenar el historial (más eficientes que vectores en Python)
    ts = [t]
    xs = [X]
    
    while t < Tmax:
        # Cálculo de propensiones
        a1 = k1
        a2 = k2 * X
        a0 = a1 + a2
        
        # Seguridad: Si la propensión total es 0, el sistema se detiene
        if a0 == 0:
            break
            
        # 1. Generar tiempo hasta el próximo evento (tau)
        # R: -log(runif(1)) / a0 
        # Esto es matemáticamente generar una variable aleatoria Exponencial con tasa a0
        tau = np.random.exponential(scale=1/a0)
        
        # 2. Determinar qué evento ocurre
        r2 = np.random.uniform()
        
        if r2 * a0 < a1:
            # Evento 1: Nacimiento / Producción
            X += 1
        else:
            # Evento 2: Muerte / Degradación
            X = max(0, X - 1)
            
        # Actualizar tiempo y almacenar estado
        t += tau
        ts.append(t)
        xs.append(X)
        
    # Devolvemos un DataFrame de Pandas 
    return pd.DataFrame({'time': ts, 'X': xs})

df_result = gillespie(Tmax=50)

# Mostrar las primeras 5 filas
print(df_result.head())

       time  X
0  0.000000  0
1  0.046275  1
2  0.068217  2
3  0.411522  3
4  0.426256  4


### Parte Integradora (Opcional)

Combinar dos modelos (ejemplo):

- NHPP → pacientes
- M/M/1 → atención
- Gillespie → biomarcadores

Evaluar: % de pacientes cuyo biomarcador excede un umbral antes de ser atendi-
dos.

### Formato y entrega

- Informe PDF (generado desde este Quarto)
- Código R/Python
- Gráficos
- Discusión de supuestos
- Semillas fijas
- Comparaciones teóricas vs simuladas
- Fecha de entrega: *20 de diciembre de 2025*

<a id="resolucion"></a>
---
# RESOLUCIÓN

### Parte 1 - Generación de Números Pseudoaleatorios y Monte Carlo

### Parte 2 - Metropolis–Hastings y Bayesian Inference

### Parte 3 - Simulación de Eventos Discretos (M/M/1 o M/M/c)

### Parte 4 - Modelos Continuos

### Parte 5 - Gillespie SSA o Next Reaction Method

### Parte Integradora